# PaaS環境構築用

## プロジェクト名の記入
プロジェクト名を記入してください

In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import clear_output

name = widgets.Text()
name

A Jupyter Widget

## 配備対象ホスト(IPアドレス)の記入

配備先のIPアドレスを入力してください

In [2]:
import ipaddress

ipaddr = widgets.Text()
ipaddr

A Jupyter Widget

## 入力されたアドレスのチェック

チェックボタンを押下して、入力したIPアドレスを確認してください

In [3]:
def on_check_button_clicked(b):
    clear_output(wait=True)
    # print("[DEBUG]: IP = %s" % ipaddr.value)
    try:
        ipaddress.ip_network(ipaddr.value)
        print("OK: 入力されたIPアドレスは正常です")
    except ValueError as e:
        print("ERROR: 無効なIPアドレスです")
    display(check_button)

check_button = widgets.Button(
    description='Check',
    disabled=False,
    button_style='INFO',
    tooltip='指定したIPアドレスを確認します',
    icon='check'
)
display(check_button)
check_button.on_click(on_check_button_clicked)

A Jupyter Widget

## git管理ツール選択
使用するgit管理ツールを選択してください

In [4]:
git = widgets.Select(
    options=['gitlab', 'gitbucket'],
    value='gitlab',
    description='Git:',
    disabled=False
)

display(git)

A Jupyter Widget

## Jenkinsの選択

Jenkinsを利用するか選択してください。

In [5]:
jenkins = widgets.Checkbox(
    value=False,
    description='Jekinsを利用する',
    disabled=False
)

jenkins

A Jupyter Widget

## 選択の確認


In [6]:
import sys
from IPython.display import clear_output

def on_confirm_button_clicked(b):
    res = "プロジェクト名:{prj}\n配備対象:{ip}\n{git_type}を利用する\nJenkinsを利用{t_or_f}"
    result = "する" if jenkins.value else "しない"
    output = res.format(
        prj=name.value, ip=ipaddr.value, git_type=git.value, t_or_f=result)    
    clear_output(wait=True)
    sys.stdout.write("\r%s" % output)
    sys.stdout.flush()
    display(confirm_button)

confirm_button = widgets.Button(
    description='確認',
    disabled=False,
    button_style='INFO',
    tooltip='構築内容を確認します',
    icon='check'
)
display(confirm_button)
confirm_button.on_click(on_confirm_button_clicked)

A Jupyter Widget

## Notebookの作成

In [7]:
import copy
import os
import shutil
from lib import nbmerger
from IPython.display import clear_output
from operator import itemgetter


# Service Map definition
# You can add additional service into following list.
# You MUST be include following keys:
#   'name': string of a service name
#   'enabled': boolean which is selected to deploy
#   'prefix': string of a file prefix number
SVC_MAP = [
    {'name': 'gitbucket', 'enabled': False, 'prefix': ''},
    {'name': 'gitlab', 'enabled': False, 'prefix': ''},
    {'name': 'jenkins', 'enabled': False, 'prefix': ''},
]

def merge_notebooks(targets, path):
    merger = nbmerger.NbMerger(name.value, ipaddr.value)
    merger.merge(targets)
    merger.write(path)


def on_button_clicked(b):
    clear_output(wait=True)
    outdir = 'outputs/' + name.value
    services = [git.value]
    if jenkins.value:
        services.append('jenkins')
    service_list = check_services(services)
    if(os.path.exists(outdir) == False):
        os.mkdir(outdir)
        targets = [
            'templates/' + s['name'] + '.ipynb'
            for s in service_list if s['enabled']
        ]
        merge_notebooks(targets, outdir + "/01_作業手順書.ipynb")
        copy_manuals(service_list)
        print("作成が完了しました。(" + outdir +")")
    else:
        print("作成はスキップされました。指定したプロジェクトは既に存在しています。(" + outdir +")")
    display(button)
    b.disabled=True


def _get_def(svc_list, service):
    if not svc_list:
        return None
    return [s for s in svc_list if s['name'] == service][0]


def _get_idx(svc_list, service):
    if not svc_list:
        return None    
    return svc_list.index(_get_def(svc_list, service))


def check_services(services):
    svc_list = sorted(copy.deepcopy(SVC_MAP), key=itemgetter('name'))
    for counter, name in enumerate(services):
        idx = _get_idx(svc_list, name)
        svc_list[idx]['enabled'] = True
        svc_list[idx]['prefix'] = '0%s' % (counter + 1)
        counter += 1
    return svc_list


def copy_manuals(service_list):
    MIX_FORMAT = "結合手順書_[%s]-[%s].ipynb"
    SINGLE_FORMAT = "個別初期設定手順書_[%s].ipynb"
    src_dir = "manuals/"
    dst_dir = "outputs/" + name.value + "/"

    for svc in service_list:
        if not svc['enabled']:
            continue
        src_file = SINGLE_FORMAT % svc['name']
        src = src_dir + src_file
        single_prefix = '02_' + svc['prefix'] + '_'
        dest = dst_dir + single_prefix + src_file
        shutil.copyfile(src, dest)

        jenkins = _get_def(service_list, 'jenkins')

        if not jenkins['enabled'] or svc['name'] == 'jenkins':
            continue

        src_file = MIX_FORMAT % (svc['name'], jenkins['name'])
        src = src_dir + src_file
        mix_prefix = '03_' + svc['prefix'] + '-' + jenkins['prefix'] + '_'
        dest = dst_dir + mix_prefix + src_file
        shutil.copyfile(src, dest)


button = widgets.Button(
    description='作成',
    disabled=False,
    button_style='INFO',
    tooltip='notebookを作成します',
    icon='check'
)
display(button)
button.on_click(on_button_clicked)

A Jupyter Widget

## (暫定)リセット
実行を一度押すと、Disableにして同じ設定で押せないようにするが、それをキャンセルするボタン。
本当は設定変更等でリセットした方がいいので暫定対応


outputをクリアする方法は不明。。



In [8]:
def on_button_enabled_clicked(b):
    button.disabled=False
    
reset_button = widgets.Button(
    description='リセット',
    disabled=False,
    button_style='INFO',
    tooltip='作成ボタンを有効化します',
    icon='check'
)
display(reset_button)
reset_button.on_click(on_button_enabled_clicked)


A Jupyter Widget